In [3]:
#imports
import numpy as np
import pandas as pd
from pathlib import Path
import glob
import os 
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import TensorBoard


In [4]:
#loading dataset using panda 
#this loads one specific csv file
#dataset = pd.read_csv(r'C:\Users\benja\OneDrive\Dokumente\Wichtiges\Projects\stock_price_predictor\dataset\archive\Data\ETFs\aadr.us.txt')
#dataset.head()
#recursively load all csv files into a list
#sets my working directory so the glob can fetch all the files
os.chdir(r'C:\Users\Admin\OneDrive\Dokumente\Wichtiges\Projects\stock_price_predictor\dataset\archive\Data')
file_list = glob.glob('**/*.*',recursive=True)
dataset = pd.DataFrame()
#merges every dataframe in one big dataframe and check for empty files
for file in file_list:
    try:
        dataset = dataset.append(pd.read_csv(file))
    except:
        print("empty file")
        continue


empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file
empty file


Data Preprocessing

In [27]:

#order the dataset by date
dataset = dataset.sort_values(by=['Date'])
#only use the date and the adjusted close price
dataset = dataset[['Date','Adj Close']]
#rescale the data to -1-1
scaler = MinMaxScaler(feature_range=(-1,1))
dataset = scaler.fit_transform(dataset)


AttributeError: 'numpy.ndarray' object has no attribute 'sort_values'

In [6]:
#use tensorboard to visualize the training process
tensorboard = TensorBoard(log_dir='logs/{}'.format('SPP'))

Splitting data into training, test, validation set

In [7]:
#function for splitting the data since the predicted label is always the stock price for of the "next day"
def split_data(data,look_back):
    x = []
    y = []
    for i in range(len(data)-look_back-1):
        x.append(data[i:(i+look_back),:])
        y.append(data[i+look_back,:])
    return np.array(x),np.array(y)
def split_sets(x,y,x_split):
    x_train = x[:x_split]
    y_train = y[:x_split]
    x_test = x[x_split:]
    y_test = y[x_split:]
    return x_train,y_train,x_test,y_test


In [9]:
x,y = split_data(dataset,2)
#define training split --> 80% training 20% testing
training_split = 0.8
#get the index where the data should be splitted
x_split = int(np.ceil(len(x)*training_split))
#split the data into training and testing data
x_train,y_train,x_test,y_test = split_sets(x,y,x_split)
#split the training data into training and validation data with a 80/20 split
x_train,x_val,y_train,y_val = split_sets(x_train,y_train,int(np.ceil(len(x_train)*0.8)))

In [20]:
#visualize the data
print(x_train[0])

[[-1.         -1.         -1.         -1.         -0.9978747  -1.        ]
 [-0.99999999 -0.99999999 -0.99999999 -0.99999999 -0.9996146  -1.        ]]


Define model

In [24]:
#defining the model
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(x_train.shape[1],x_train.shape[2])))
#dropout for regularization --> randomly drops 50% of the neurons
model.add(Dropout(0.5))
model.add(LSTM(units = 50,return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(units = 50))
model.add(Dropout(0.5))
#output layer --> fully connected layer
model.add(Dense(y_train.shape[1]))
#compiling the model
model.compile(optimizer='adam',loss='mean_squared_error')
#for description of the model --> none = dimnension is variable
model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 2, 50)             11400     
                                                                 
 dropout_3 (Dropout)         (None, 2, 50)             0         
                                                                 
 lstm_4 (LSTM)               (None, 2, 50)             20200     
                                                                 
 dropout_4 (Dropout)         (None, 2, 50)             0         
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_5 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                

train the model 

In [23]:
#training the model
#overfitting model on one sample of the data
model.fit(x_train[0].reshape(1,x_train.shape[1],x_train.shape[2]),y_train[0].reshape(1,y_train.shape[1]),epochs=100,batch_size=1,verbose=1,callbacks=[tensorboard])


ValueError: cannot reshape array of size 12 into shape (1,2)

train model 

In [66]:
#train the model on the training set
model.fit(x_train,y_train,validation_data=(x_val,y_val),batch_size=32,epochs=1,callbacks=[tensorboard])

436331/436331 [==============================] - 2631s 6ms/step - loss: 3.4827e-05 - val_loss: 1.2068e-06


In [67]:
#save the model --> current model SPP.h5
model.save('SPP.h5')

In [ ]:
#load the model
model = tf.keras.models.load_model('SPP.h5')

In [68]:
#testing the model
#predicting the stock price for the next day
y_pred = model.predict(x_test)
#rescale the data back to the original scale
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)
#calculate the error
error = np.mean(np.abs(y_pred-y_test))
#visualize the results
plt.plot(y_pred)
plt.plot(y_test)
plt.show()


76358/76358 [==============================] - 172s 2ms/step
146690.4358881029
